In [1]:
import queue 

In [2]:
p = queue.PriorityQueue()

In [67]:
frequencies = {'a':0.1,'b':0.15, 'c':0.2, 'd':0.25, 'e':0.3 }

In [72]:
symbol_code = {}

In [59]:
def fill_priorityQueue(elem_by_frequency):
    symbol_by_code = {}
    for key, value in elem_by_frequency.items():
        symbol_code[key]='';
        p.put((value, key))
    return symbol_by_code

In [73]:
#fill prority queue and code dictionary
symbol_code = fill_priorityQueue(frequencies)

In [69]:
# chack if empty
print(p.empty())

False


In [63]:
def add_bits_key(key, bit, symbol_code):
    if len(key) == 1:
        symbol_code[key] = bit
    else:
        for symbol in key:
            symbol_code[symbol] = bit + symbol_code[symbol]

In [64]:
def build_code(symbol_code):
    while p.qsize() > 1:
        right = p.get();
        left = p.get();
        if right is None or left is None:
            break;
        left_key = left[1]
        left_vaalue = left[0]
        right_key = right[1]
        right_value = right[0]
        
        add_bits_key(left_key,'1',symbol_code)
        add_bits_key(right_key,'0', symbol_code)
        print(str(right_value + left_vaalue)+"->"+left_key + right_key)
        
        p.put((right_value + left_vaalue, left_key + right_key))
        

In [75]:
build_code(symbol_code)

0.25->ba
0.45->bac
0.55->ed
1.0->edbac


In [57]:
while True:    
    if p.empty() == True:
        break;
    current = p.get()
    print(str(current[0])+ "->" + str(current[1]))

1.4->cbcbaedf


In [18]:
for let in 'ght':
    print(let)


g
h
t


In [76]:
print(symbol_code)
print (frequencies)

{'b': '100', 'a': '101', 'c': '11', 'e': '00', 'd': '01'}
{'a': 0.1, 'b': 0.15, 'c': 0.2, 'd': 0.25, 'e': 0.3}
